In [ ]:
Constants KingJohn; 2; UCB; : : :
Predicates Brother; &gt;; : : :
Functions Sqrt; LeftLegOf; : : :
Variables x; y; a; b; : : :
Connectives ^ _ : ) ,
Equality =
Quantifiers Universal , Existential

Objects - people
• Two unary predicates - Male and Female
• Kinship relations—parenthood, brotherhood, marriage, and so on—represented by binary predicates:
 Parent, Sibling, Brother , Sister , Child , Daughter, Son, Spouse, Wife,
Husband, Grandparent , Grandchild , Cousin, Aunt, and Uncle

• Functions - Mother and Father

In [1]:
import re

def classify_fol(fol_sentence):
    constants = []
    variables = []
    predicates = []
    functions = []
    quantifiers = []
    connectives = []
    implications = []
    constants_pattern = r"\b[A-Z][a-zA-Z0-9]*\b"
    variables_pattern = r"\b[a-z]\b"
    predicates_functions_pattern = r"\b([A-Za-z]+)\(([a-zA-Z0-9, ]+)\)"
    quantifiers_pattern = r"(∀|∃)"
    connectives_pattern = r"(and|or|not|→|∨|∧)"


    constants = re.findall(constants_pattern, fol_sentence)
    

    variables = re.findall(variables_pattern, fol_sentence)

    matches = re.findall(predicates_functions_pattern, fol_sentence)
    for match in matches:
        func_name = match[0]
        params = match[1].split(',')
        functions.append(func_name)
        predicates.append(func_name) 
        
        for param in params:
            param = param.strip()
            if re.match(constants_pattern, param):
                constants.append(param)
            elif re.match(variables_pattern, param):
                variables.append(param)

    quantifiers = re.findall(quantifiers_pattern, fol_sentence)

    connectives = re.findall(connectives_pattern, fol_sentence)

    implications = re.findall(r"→", fol_sentence)

    result = {
        "constants": list(set(constants)), 
        "variables": list(set(variables)), 
        "predicates": list(set(predicates)),
        "functions": list(set(functions)),
        "quantifiers": list(set(quantifiers)),
        "connectives": list(set(connectives)),
        "implications": len(implications) > 0
    }
    
    return result



def convert_to_fol(sentence):
    sentence = sentence.strip().lower()

    fol_rules = {
        r"(.+) was a (.+)\.": lambda subject, predicate: f"{predicate.capitalize()}({subject.capitalize()})",
        r"all (.+) were (.+)\.": lambda group1, group2: f"∀x ({group1.capitalize()}(x) → {group2.capitalize()}(x))",
        r"everyone is legal to someone\.": lambda: "∀x ∃y Legal(x, y)",
        r"(.+) tried to (.+) (.+)\.": lambda subject, action, target: f"Tried{action.capitalize()}({subject.capitalize()}, {target.capitalize()})",
        r"all men are people\.": lambda: "∀x (Man(x) → Person(x))",
        r"all romans were loyal to (.+) or hated him\.": lambda target: f"∀x (Roman(x) → (LoyalTo(x, {target.capitalize()}) ∨ Hate(x, {target.capitalize()})))",
        r"people only try to (.+) if they are not loyal to\." : lambda action: f"∀x ∀y (Person(x) ∧ Try{action.capitalize()}(x, y) → ¬LoyalTo(x, y))"
    }
    

    for pattern, fol_lambda in fol_rules.items():
        match = re.match(pattern, sentence)
        if match:
            return fol_lambda(*match.groups())
        
    
    return "No FOL conversion rule found."

sentences = [
    "Marcus was a man.",
    "Marcus was a Pompian.",
    "All Pompians were Romans.",
    "Caeser was a ruler.",
    "Everyone is legal to someone.",
    "Marcus tried to assassinate Caeser.",
    "All men are people.",
    "All Romans were loyal to Caeser or hated him.",
    "People only try to assassinate if they are not loyal to."
]

for sentence in sentences:
    fol_sentence = convert_to_fol(sentence)
    print(f"\n\nOriginal: {sentence}\nFOL: {fol_sentence}\nIt has")
    print(classify_fol(fol_sentence))




Original: Marcus was a man.
FOL: Man(Marcus)
It has
{'constants': ['Marcus', 'Man'], 'variables': [], 'predicates': ['Man'], 'functions': ['Man'], 'quantifiers': [], 'connectives': [], 'implications': False}


Original: Marcus was a Pompian.
FOL: Pompian(Marcus)
It has
{'constants': ['Pompian', 'Marcus'], 'variables': [], 'predicates': ['Pompian'], 'functions': ['Pompian'], 'quantifiers': [], 'connectives': [], 'implications': False}


Original: All Pompians were Romans.
FOL: ∀x (Pompians(x) → Romans(x))
It has
{'constants': ['Romans', 'Pompians'], 'variables': ['x'], 'predicates': ['Romans', 'Pompians'], 'functions': ['Romans', 'Pompians'], 'quantifiers': ['∀'], 'connectives': ['→'], 'implications': True}


Original: Caeser was a ruler.
FOL: Ruler(Caeser)
It has
{'constants': ['Ruler', 'Caeser'], 'variables': [], 'predicates': ['Ruler'], 'functions': ['Ruler'], 'quantifiers': [], 'connectives': [], 'implications': False}


Original: Everyone is legal to someone.
FOL: ∀x ∃y Legal(x, 

In [ ]:
class Constant:
    def __init__(self, name):
        self.name = str(name)
        
    def __repr__(self):
        return self.name

class Predicate:
    def __init__(self, name, *args):
        self.name = name
        self.args = args
        
    def __repr__(self):
        return f"{self.name}({', '.join(map(str, self.args))})"

class Quantifier:
    def __init__(self, quantifier, variable, statement):
        self.quantifier = quantifier
        self.variable = variable
        self.statement = statement
        
    def __repr__(self):
        return f"{self.quantifier}{self.variable} ({self.statement})"

class LogicalConnective:
    def __init__(self, connective, left, right):
        self.connective = connective
        self.left = left
        self.right = right
        
    def __repr__(self):
        return f"({self.left} {self.connective} {self.right})"


Jack = Constant("Jack")
Curiosity = Constant("Curiosity")
Tuna = Constant("Tuna")

def animal(y):
    return Predicate("Animal", y)

def loves(x, y):
    return Predicate("Loves", x, y)

def kills(x, y):
    return Predicate("Kills", x, y)


everyone_loves_animals = Quantifier("∀", "x",
    LogicalConnective("→",
        Quantifier("∀", "y", LogicalConnective("→", animal(Constant("y")), loves(Constant("x"), Constant("y")))),
        Quantifier("∃", "z", loves(Constant("z"), Constant("x")))
    )
)


anyone_kills_animal = Quantifier("∀", "x",
    LogicalConnective("→",
        Quantifier("∃", "y", LogicalConnective("∧", animal(Constant("y")), kills(Constant("x"), Constant("y")))),
        Quantifier("∀", "z", Predicate("¬", loves(Constant("z"), Constant("x"))))
    )
)

jack_loves_all_animals = Quantifier("∀", "y", loves(Jack, Constant("y")))

either_jack_or_curiosity_killed = LogicalConnective("∨", kills(Jack, Tuna), kills(Curiosity, Tuna))

did_curiosity_kill_cat = Predicate('¬',kills(Curiosity, Tuna))

tunacatis_animal=Quantifier('∀','x',LogicalConnective('→',Predicate('Cat',Constant('x')),animal(Constant('x'))))

print("1. Everyone who loves all animals is loved by someone:\n", everyone_loves_animals)
print("2. Anyone who kills an animal is loved by no one:\n", anyone_kills_animal)
print("3. Jack loves all animals:\n", jack_loves_all_animals)
print("4. Either Jack or Curiosity killed the cat named Tuna:\n", either_jack_or_curiosity_killed)
print("5. Did Curiosity kill the cat?\n", did_curiosity_kill_cat)
print("6",Predicate('Cat',Constant('Tuna')))
print("7",tunacatis_animal)